In [2]:
!pip install torchviz

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.0/47.0 kB 1.7 MB/s eta 0:00:00
  Created wheel for torchviz: filename=torchviz-0.0.2-py3-none-any.whl size=4131 sha256=c21f0052189212990de2773c872e9852d12cce14f8ab76774076ba52bc9ef058
  Stored in directory: /Users/vinoth/Library/Caches/pip/wheels/29/65/6e/db2515eb1dc760fecd36b40d54df65c1e18534013f1c037e2e
Successfully built torchviz


In [7]:
!conda install -c anaconda graphviz --yes

^C

CondaError: KeyboardInterrupt



In [12]:
!pip install graphviz

In [13]:
from torchviz import make_dot
import torch
import torch.nn.functional as F
from torch.nn import BatchNorm1d, Dropout, Linear
from torch_geometric.nn import GCNConv, global_mean_pool

class Net(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(Net, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)
        self.bn1 = BatchNorm1d(16)
        self.conv2 = GCNConv(16, 32)
        self.bn2 = BatchNorm1d(32)
        self.fc = Linear(32, num_classes)
        self.dropout = Dropout(0.5)

    def forward(self, data):
        x, edge_index, batch = data['x'], data['edge_index'], data['batch']
        x = self.conv1(x, edge_index)
        x = self.bn1(x)
        x = F.relu(x)
        x = self.dropout(x)
        x = self.conv2(x, edge_index)
        x = self.bn2(x)
        x = global_mean_pool(x, batch)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)


# Define example input data (modify this as per your actual input data)
data = {
    'x': torch.randn(10, 5),
    'edge_index': torch.randint(10, (2, 20)),
    'batch': torch.tensor([0, 0, 0, 1, 1, 1, 2, 2, 2, 2])
}

num_node_features = data['x'].size(1)
num_classes = 3

model = Net(num_node_features, num_classes)

# Visualize the graph
make_dot(model(data), params=dict(model.named_parameters())).render("model", format="png")


ExecutableNotFound: failed to execute PosixPath('dot'), make sure the Graphviz executables are on your systems' PATH